# Convert verse ranges of genres to TF verse node features

In [2]:
import collections
import pandas as pd
from tf.fabric import Fabric
from tf.compose import modify
from tf.app import use
A = use('bhsa', hoist=globals())

In [20]:
genre_ranges = pd.read_csv('genre_ranges.csv')

In [21]:
genre_ranges

,book,ch start,vs start,ch end,vs end,genre
0,Genesis,1,1,4,16,prose
1,Genesis,4,17,4,23,list
2,Genesis,4,24,4,26,prose
3,Genesis,5,1,5,32,list
4,Genesis,6,1,9,29,prose
...,...,...,...,...,...,...
215,1_Chronicles,16,8,16,36,poetry
216,1_Chronicles,16,37,23,1,prose
217,1_Chronicles,23,2,27,34,list
218,1_Chronicles,28,1,29,30,prose


# Compile data & sanity checks

In [22]:
# check book values
genre_ranges.book.unique()

array(['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy',
       'Joshua', 'Judges', '1_Samuel', '2_Samuel', '1_Kings', '2_Kings',
       'Isaiah', 'Jeremiah', 'Ezekiel', 'Hosea', 'Joel', 'Amos',
       'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk', 'Zephaniah',
       'Haggai', 'Zechariah', 'Malachi', 'Psalms', 'Job', 'Proverbs',
       'Ruth', 'Song_of_songs', 'Ecclesiastes', 'Lamentations', 'Esther',
       'Daniel', 'Ezra', 'Nehemiah', '1_Chronicles', '2_Chronicles'],
      dtype=object)

In [23]:
# check genre values
genre_ranges.genre.unique()

array(['prose', 'list', 'poetry', 'instruction', 'prophetic'],
      dtype=object)

In [24]:
# check book name alignment with BHSA english names
for book in genre_ranges.book.unique():
    bhsa_node = T.nodeFromSection((book,))
    if not bhsa_node:
        raise Exception(book)

In [25]:
def verse_node_range(start, end, tf_api):
    """Generate a list of verse nodes for a given range of reference tuples.
    
    Note that start and end are both inclusive bounds.
    
    Args:
        start: 3-tuple of (book, n_ch, n_vs)
        end: 3-tuple of (book, n_ch, n_vs)
    Returns:
        list of nodes
    """
    start_node = tf_api.T.nodeFromSection(start)
    end_node = tf_api.T.nodeFromSection(end)
    nodes = [start_node]
    while nodes[-1] < end_node:
        nodes.append(tf_api.L.n(nodes[-1],'verse')[0])
    return nodes

In [26]:
# check for missing verses
# or double-counted verses

verse2genre = {} # will be used for TF export
verse2count = collections.Counter()

for book, startch, startvs, endch, endvs, genre in genre_ranges.values:
    start = (book, startch, startvs)
    end = (book, endch, endvs)
    for verse in verse_node_range(start, end, A.api):
        verse2genre[verse] = genre
        verse2count[verse] += 1
        
# check for double-labeled verses
for verse,count in verse2count.items():
    if count > 1:
        print(verse, T.sectionFromNode(verse))

In [28]:
# check for missing verses
all_verses = set(F.otype.s('verse'))
for missing_verse in (all_verses - set(verse2genre.keys())):
    print(missing_verse, T.sectionFromNode(missing_verse))

In [30]:
#verse2genre

# Export TF Features

In [50]:
nodeFeatures = {'genre': verse2genre}
featureMeta = {
    'genre': {
        'description': '(sub)genre of a verse node',
        'authors': 'Dirk Bakker, Marianne Kaajan, Martijn Naaijer, Wido van Peursen, Janet Dyk',
        'origin': 'the genre feature was tagged during the NWO-funded syntactic variation project (2013-2018) of the ETCBC, VU Amsterdam',
        'source_URL': 'https://github.com/MartijnNaaijer/phdthesis/blob/master/Various/subgenres_synvar.xls',
        'valueType': 'str',
    } 
}

In [51]:
TF = Fabric('tf/c')
TF.save(nodeFeatures=nodeFeatures, metaData=featureMeta)

This is Text-Fabric 8.3.2
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

1 features found and 0 ignored
  0.00s Warp feature "otype" not found in
tf/c/
  0.00s Warp feature "oslots" not found in
tf/c/
  0.01s Warp feature "otext" not found. Working without Text-API

  0.00s Exporting 1 node and 0 edge and 0 config features to tf/c:
   |     0.04s T genre                to tf/c
  0.04s Exported 1 node features and 0 edge features and 0 config features to tf/c


True

## Tests

In [3]:
TF = Fabric(locations=['~/github/etcbc/bhsa/tf/c', 'tf/c'])
API = TF.load('genre')
API.makeAvailableIn(globals())

This is Text-Fabric 8.3.2
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

115 features found and 0 ignored
  0.00s loading features ...
   |     0.00s Dataset without structure sections in otext:no structure functions in the T-API
  4.09s All features loaded/computed - for details use loadLog()


[('Computed',
  'computed-data',
  ('C Computed', 'Call AllComputeds', 'Cs ComputedString')),
 ('Features', 'edge-features', ('E Edge', 'Eall AllEdges', 'Es EdgeString')),
 ('Fabric', 'loading', ('TF',)),
 ('Locality', 'locality', ('L Locality',)),
 ('Nodes', 'navigating-nodes', ('N Nodes',)),
 ('Features',
  'node-features',
  ('F Feature', 'Fall AllFeatures', 'Fs FeatureString')),
 ('Search', 'search', ('S Search',)),
 ('Text', 'text', ('T Text',))]

In [5]:
F.otype.s('verse')

range(1414354, 1437567)

In [10]:
verse_data = []

for verse_n in F.otype.s('verse'):
    genre = F.genre.v(verse_n)
    book, chapter, verse = T.sectionFromNode(verse_n)
    ref = f'{book[:3]} {chapter}:{verse}'
    verse_data.append({
        'node': verse_n,
        'ref': ref,
        'book': book,
        'genre': genre,
        'text': T.text(verse_n),
    })
    
verse_df = pd.DataFrame(verse_data)
verse_df.set_index('node', inplace=True)
verse_df.head()

,ref,book,genre,text
node,,,,
1414354,Gen 1:1,Genesis,prose,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖י...
1414355,Gen 1:2,Genesis,prose,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁך...
1414356,Gen 1:3,Genesis,prose,וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽ...
1414357,Gen 1:4,Genesis,prose,וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַי...
1414358,Gen 1:5,Genesis,prose,וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שׁ...


In [11]:
verse_df.genre.value_counts()

prose          9337
poetry         5286
prophetic      4615
instruction    2319
list           1656
Name: genre, dtype: int64

In [13]:
verse_df[verse_df.genre == 'prophetic'].book.value_counts()

Jeremiah     1296
Isaiah       1228
Ezekiel      1082
Zechariah     211
Hosea         197
Amos          146
Micah         105
Joel           73
Habakkuk       56
Malachi        55
Zephaniah      53
Nahum          47
Haggai         38
Obadiah        21
2_Kings         7
Name: book, dtype: int64

In [14]:
verse_df[verse_df.genre == 'list'].book.value_counts()

1_Chronicles    619
Numbers         307
Joshua          222
Nehemiah        184
Genesis         144
Ezra            116
1_Kings          20
Exodus           19
2_Samuel         16
Esther            5
1_Samuel          4
Name: book, dtype: int64

In [23]:
# How many verses per book are a given genre?

book2genre = pd.pivot_table(
    verse_df,
    index='book',
    columns=['genre'],
    aggfunc='size',
    fill_value=0,
)

book2genre

genre,instruction,list,poetry,prophetic,prose
book,,,,,
1_Chronicles,0,619,29,0,295
1_Kings,0,20,0,0,797
1_Samuel,0,4,9,0,798
2_Chronicles,0,0,0,0,822
2_Kings,0,0,0,7,712
2_Samuel,0,16,64,0,615
Amos,0,0,0,146,0
Daniel,0,0,11,0,346
Deuteronomy,342,0,70,0,547


In [25]:
# get percentages

book2genre.div(book2genre.sum(1), 0)

genre,instruction,list,poetry,prophetic,prose
book,,,,,
1_Chronicles,0.000000,0.656416,0.030753,0.000000,0.312831
1_Kings,0.000000,0.024480,0.000000,0.000000,0.975520
1_Samuel,0.000000,0.004932,0.011097,0.000000,0.983970
2_Chronicles,0.000000,0.000000,0.000000,0.000000,1.000000
2_Kings,0.000000,0.000000,0.000000,0.009736,0.990264
2_Samuel,0.000000,0.023022,0.092086,0.000000,0.884892
Amos,0.000000,0.000000,0.000000,1.000000,0.000000
Daniel,0.000000,0.000000,0.030812,0.000000,0.969188
Deuteronomy,0.356621,0.000000,0.072993,0.000000,0.570386
